In [1]:
import sys
sys.path.append("/home/yuahe/projects/OntoAlign-py")
from ontoalign.embeds import PretrainedBert
from ontoalign.onto import Ontology
import torch
from sklearn.metrics.pairwise import cosine_similarity

bert = PretrainedBert("emilyalsentzer/Bio_ClinicalBERT")

Load the Pretrained BERT model...


Check if the bert batch embeddings is equivalent to sentence by sentence

In [2]:
batch_sents = ["donald trump", "donald j trump", "us president", "apple tree", "pineapple fruits", "he is a man", "he is a male"]

In [3]:
batch_embeds, _ = bert.batch_word_embeds(batch_sents, -2)

In [4]:
for i in range(len(batch_sents)):
    sent = [batch_sents[i]]
    word_embeds = bert.batch_word_embeds(sent, -2)[0].squeeze(0)
    sent_len = word_embeds.shape[0]
    # display(word_embeds.shape, batch_embeds[i].shape)
    print(torch.sum(batch_embeds[i][:sent_len] - word_embeds), cosine_similarity(batch_embeds[i][:sent_len], word_embeds).diagonal().mean())

tensor(-4.5709e-06) 1.0
tensor(9.2884e-07) 1.0
tensor(6.7205e-06) 0.99999994
tensor(-1.0286e-05) 1.0
tensor(-1.9894e-05) 0.9999998
tensor(5.5869e-06) 0.99999976
tensor(2.9111e-05) 1.0


In [5]:
batch_sent_embeds = bert.batch_sent_embeds_last_2_mean(batch_sents)

In [6]:
for i in range(len(batch_sents)):
    sent = [batch_sents[i]]
    word_embeds = bert.batch_word_embeds(sent, -2)[0].squeeze(0)
    sent_embed = torch.mean(word_embeds, dim=0)
    sent_len = word_embeds.shape[0]
    print(torch.sum(batch_sent_embeds[i] - sent_embed), cosine_similarity(batch_sent_embeds[i].unsqueeze(0), sent_embed.unsqueeze(0)).diagonal().mean())

tensor(-1.1625e-06, dtype=torch.float64) 0.9999999999999678
tensor(-6.7429e-07, dtype=torch.float64) 0.99999999999996
tensor(2.2473e-06, dtype=torch.float64) 0.9999999999999638
tensor(-3.0763e-06, dtype=torch.float64) 0.9999999999999628
tensor(-4.1638e-06, dtype=torch.float64) 0.9999999999999739
tensor(1.6505e-06, dtype=torch.float64) 0.9999999999999691
tensor(3.6689e-06, dtype=torch.float64) 0.9999999999999681


Up to now, we see that the batch algorithm brings certain computational error but it's very small (< $10^{-4}$) 

In [7]:
batch_lexicon_sizes = [3, 1, 1, 2]
batch_lexicon_sents = ["donald trump", "donald j trump", "us president", "apple tree", "pineapple fruits", "he is a man", "he is a male"]
all_lexicon_sents_embeds = bert.batch_sent_embeds_last_2_mean(batch_lexicon_sents)

In [8]:
x1 = torch.stack([torch.mean(all_lexicon_sents_embeds[:3], dim=0),
                torch.mean(all_lexicon_sents_embeds[3:4], dim=0),
                torch.mean(all_lexicon_sents_embeds[4:5], dim=0),
                torch.mean(all_lexicon_sents_embeds[5:7], dim=0)])

In [9]:
x2 = bert.entity_embeds_from_batch_lexicon("batch_sent_embeds_last_2_mean", batch_lexicon_sizes, batch_lexicon_sents)

In [10]:
g1 = torch.mean(bert.batch_sent_embeds_last_2_mean(["donald trump", "donald j trump", "us president"]), dim=0)
g2 = torch.mean(bert.batch_sent_embeds_last_2_mean(["apple tree"]), dim=0)
g3 = torch.mean(bert.batch_sent_embeds_last_2_mean(["pineapple fruits"]), dim=0)
g4 = torch.mean(bert.batch_sent_embeds_last_2_mean(["he is a man", "he is a male"]), dim=0)
x3 = torch.stack([g1, g2, g3, g4])

In [11]:
cosine_similarity(x1, x2)

array([[1.        , 0.92867219, 0.9334201 , 0.87184914],
       [0.92867219, 1.        , 0.93917666, 0.85035664],
       [0.9334201 , 0.93917666, 1.        , 0.83408974],
       [0.87184914, 0.85035664, 0.83408974, 1.        ]])

In [12]:
cosine_similarity(x1, x3)

array([[1.        , 0.92867218, 0.93342009, 0.87184915],
       [0.92867218, 1.        , 0.93917666, 0.85035664],
       [0.93342009, 0.93917665, 1.        , 0.83408974],
       [0.87184914, 0.85035662, 0.83408973, 1.        ]])

In [13]:
cosine_similarity(x2, x3)

array([[1.        , 0.92867218, 0.93342009, 0.87184915],
       [0.92867218, 1.        , 0.93917666, 0.85035664],
       [0.93342009, 0.93917665, 1.        , 0.83408974],
       [0.87184914, 0.85035662, 0.83408973, 1.        ]])

Up to this point, the batch lexicon embeddings is settled.